In [ ]:
import logging, os, json, glob, h5py, pickle, math, collections
import numpy as np
from einops import rearrange, repeat
import tqdm.notebook as tqdm

import torch
from torch import nn
import torch.nn.functional as F

from pathlib import Path
from typing import Callable, Optional, Tuple, Union, Any

def load_h5_file(file_path: Union[str, Path], sl: Optional[slice] = None, to_torch: bool = False) -> np.ndarray:
    """Given a file path to an h5 file assumed to house a tensor, load that
    tensor into memory and return a pointer.

    Parameters
    ----------
    file_path: str
        h5 file to load
    sl: Optional[slice]
        slice to load (data is written in chunks for faster access to rows).
    """
    # load
    with h5py.File(str(file_path) if isinstance(file_path, Path) else file_path, "r") as fr:
        data = fr.get("array")
        if sl is not None:
            data = np.array(data[sl])
        else:
            data = np.array(data)
        if to_torch:
            data = torch.from_numpy(data)
            data = data.to(dtype=torch.float)
        return data
def write_data_to_h5(data: np.ndarray, filename: Union[str, Path], compression="gzip", compression_level=9, dtype="uint8", verbose=True):
    """write data in gzipped h5 format.

    Parameters
    ----------
    data
    filename
    compression
    compression_level
    verbose
    """
    with h5py.File(filename if isinstance(filename, str) else str(filename), "w", libver="latest") as f:
        if data.dtype != dtype:
            logging.warning(f"Found data with {data.dtype}, expected {dtype}.")
        if verbose:
            print(f"writing {filename} ...")
        f.create_dataset(
            # `chunks=(1, *data.shape[1:])`: optimize for row access!
            "array",
            shape=data.shape,
            data=data,
            chunks=(1, *data.shape[1:]),
            dtype=dtype,
            compression=compression,
            compression_opts=compression_level,
        )
        if verbose:
            print(f"... done writing {filename}")

#### Make some prediction

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
device = 'cpu'
class Runner():
    def __init__(self, model_name='Resnet3D'):
        self.model_name = model_name
        self.model = globals()[model_name]()
        # if torch.cuda.device_count() > 1:
            # self.model = torch.nn.DataParallel(self.model)
        self.model.to(device)
    def predict(self, checkpoint_dir='./checkpoints/Resnet3D.pk', mode='core', use_mask=True):
        
        submission_name = f'submission_{self.model_name}'
        
        model_dic = torch.load(checkpoint_dir)
        new_state_dict = collections.OrderedDict() 
        for k, v in model_dic.items(): 
            name = k.replace('module.', '')# remove `module.` 
            new_state_dict[name] = v
        self.model.load_state_dict(new_state_dict)
        self.model.eval()
        
        if mode =='core':
            cities = ['BERLIN', 'CHICAGO', 'ISTANBUL', 'MELBOURNE']
            postfix = 'temporal'
        else:
            cities = ['VIENNA', 'NEWYORK']
            postfix = 'spatiotemporal'
            
        for city in cities:
            print(f'Predicting {city}...')
            # read city information
            with h5py.File(f'../data/raw/{city}/{city}_test_{postfix}.h5', "r") as f:
                test_data = f.get("array")
                test_data = np.array(test_data)
            # MASK
            if use_mask:
                mask = np.zeros([495, 436, 8])
                for n in range(100):
                    for t in range(12):
                        mask = np.logical_or(mask, test_data[n, t, :, :, :])
                mask = torch.from_numpy(mask).to(device)

            y_preds = []
            for x in tqdm.tqdm(test_data):
                x = torch.from_numpy(x).unsqueeze(0).float().to(device)
                if use_mask:
                    y_pred = self.model(x) * mask
                else:
                    y_pred = self.model(x)
                y_pred[y_pred < 1] = 0
                y_pred[y_pred > 255] = 255
                y_preds.append(y_pred.cpu().detach().numpy())
                torch.cuda.empty_cache()
            y_preds = np.concatenate(y_preds, axis=0)

            Path(f'{submission_name}/{city}/').mkdir(parents=True, exist_ok=True)
            write_data_to_h5(y_preds.astype(np.uint8), f'{submission_name}/{city}/{city}_test_{postfix}.h5')
            print('Done.')

In [ ]:
# Model 1 - Resnet3D
inlen, outlen = 12, 6
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.cnn = nn.Conv3d(12, 12, kernel_size=1, stride=1, padding=0, bias=False)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
    def forward(self, x):
        """
        Args:
            x: Tensor, [1, 12, 495, 436, 8]
        """
        x = self.cnn(x)
        x = x.permute(0,2,3,1,4)
        pe = self.pe[:12].squeeze()
        #print(x.shape, pe.shape)
        x = x + pe
        x = x.permute(0,3,1,2,4)
        #print(x.shape)
        return self.dropout(x) 
class Resnet3DBlock(nn.Module):

    def __init__(self, hidden_size=4, leakyrate=0.2):
        super(Resnet3DBlock, self).__init__()
        self.conv1 = nn.Conv3d(hidden_size, hidden_size, kernel_size=(1,3,3), stride=(1,1,1), padding=(0,1,1))
        #self.conv2 = nn.Conv3d(hidden_size, hidden_size, kernel_size=(3,3,3), stride=(1,1,1), padding=(1,1,1), bias=False)
        #self.bn = nn.BatchNorm3d(hidden_size)
        self.layer_norm = nn.LayerNorm((8, 495, 436))
        #self.relu = nn.ReLU()
        self.relu = nn.LeakyReLU(leakyrate)
        #self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.layer_norm(out)
        out = self.relu(out)
        #out = self.dropout(out)

        out = (out + identity)

        return out
class InceptionIn(nn.Module):
    def __init__(self, hidden_size=24):
        super(InceptionIn, self).__init__()
        self.cnn1x1 = nn.Conv3d(inlen, hidden_size, kernel_size=(1,1,1), stride=(1,1,1), padding=(0,0,0))
        self.cnn3x3 = nn.Conv3d(inlen, hidden_size, kernel_size=(3,3,3), stride=(1,1,1), padding=(1,1,1))
        # self.cnn7x7 = nn.Conv3d(inlen, hidden_size, kernel_size=(3,7,7), stride=(1,1,1), padding=(1,3,3))
    def forward(self, x):
        out1 = self.cnn1x1(x)
        out2 = self.cnn3x3(x)
        # out3 = self.cnn7x7(x)
        out = out1 + out2 # + out3
        return out
class Resnet3D(nn.Module):

    def __init__(self, n_layers=4, hidden_size=16, leakyrate=0.2):
        super(Resnet3D, self).__init__()
        self.n_layers = n_layers
        self.pe = PositionalEncoding(8)
        self.relu = nn.LeakyReLU(leakyrate)
        #self.relu = nn.ReLU()
        #self.cnn_in = nn.Conv3d(inlen, hidden_size, kernel_size=(3,3,3), stride=(1,1,1), padding=(1,1,1), bias=True)
        self.cnn_in = InceptionIn(hidden_size)
        self.layer_norm = nn.LayerNorm((8, 495, 436))
        # self.bn = nn.BatchNorm3d(hidden_size)

        for i in range(n_layers):
            setattr(self, f'resnet{i}', Resnet3DBlock(hidden_size, leakyrate))
        self.cnn_out = nn.Conv3d(hidden_size, outlen, kernel_size=(3,3,3), stride=(1,1,1), padding=(1,1,1))

        # init
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Conv3d)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm, nn.BatchNorm3d)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # positional encoding
        x = self.pe(x)
        
        x = rearrange(x, 'b t w h c -> b t c w h')
       
        # print('input shape is ', x.shape)
        x = self.layer_norm(self.relu(self.cnn_in(x)))

        for i in range(self.n_layers):
            x = getattr(self, f'resnet{i}')(x)

        #print("out in",x.shape)
        x = F.relu(self.cnn_out(x))

        #x[x > 255.0] = 255.0
        #x[x < 1.0] = 0

        x = rearrange(x, 'b t c w h -> b t w h c')
        return x

runner = Runner(model_name='Resnet3D')
runner.predict(checkpoint_dir='./checkpoints/Resnet3D.pk', mode='core')

In [ ]:
# Model 2 - SparseUNet
import MinkowskiEngine as ME
import MinkowskiEngine.MinkowskiFunctional as MF
class SparseUNet(ME.MinkowskiNetwork):
    def __init__(self, hs_block1=12, hs_block2=12, hs_block3=16, block3_tr=8, block2_tr=48):
        in_nchannel, out_nchannel, D = 12, 6, 3
        super(SparseUNet, self).__init__(D)
        self.block1 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=in_nchannel,
                out_channels=hs_block1,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(hs_block1),
        )
        self.block2 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=hs_block1,
                out_channels=hs_block2,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(hs_block2),
        )
        self.block3 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=hs_block2,
                out_channels=hs_block3,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(hs_block3),
        )
        self.block3_tr = torch.nn.Sequential(
            ME.MinkowskiConvolutionTranspose(
                in_channels=hs_block3,
                out_channels=block3_tr,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(block3_tr),
        )
        self.block2_tr = torch.nn.Sequential(
            ME.MinkowskiConvolutionTranspose(
                in_channels=hs_block2+block3_tr,
                out_channels=block2_tr,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(block2_tr),
        )
        self.conv1_tr = ME.MinkowskiConvolution(
            in_channels=hs_block1+block2_tr,
            out_channels=out_nchannel,
            kernel_size=1,
            stride=1,
            dimension=D,
            expand_coordinates=True
        )

    def forward(self, x):
        # x to sparse tensor
        x = ME.MinkowskiOps.to_sparse(x)

        out_s1 = self.block1(x)
        out = MF.relu(out_s1)

        out_s2 = self.block2(out)
        out = MF.relu(out_s2)

        out_s4 = self.block3(out)
        out = MF.relu(out_s4)

        out = MF.relu(self.block3_tr(out))
        out = ME.cat(out, out_s2)
        out = MF.relu(self.block2_tr(out))
        out = ME.cat(out, out_s1)

        out = self.conv1_tr(out)

        dense_output, min_coord, tensor_stride = out.dense()
        missed_min_coordinate = np.array(dense_output.size()[-3:]) - np.array([495, 436, 8])
        if missed_min_coordinate.sum() != 0:
            dense_output, min_coord, tensor_stride = out.dense(
            min_coordinate=torch.IntTensor(missed_min_coordinate)
        )
        return dense_output

runner = Runner(model_name='SparseUNet')
runner.predict(checkpoint_dir='./checkpoints/SparseUNet.pk', mode='extended')